# Assignment 8: Transparency of Bernoulli naive Bayes

Jianwei Qian A20346099

## 1. Data set
Statlog (German Credit Data) Data Set. <https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)>

This dataset classifies people as with good or bad credit based on a set of attributes. The features include Balance of checking/saving account, Duration in month, Credit history, Credit amount, Marital status and sex, Property, Age, Housing, Job, etc. Notice ordered categorical attributes are coded as integer and unordered categorical attributes are split into multiple binary attributes.

20 features, 936 instances

**Categorical features:** 13 in total

Chckg_acct_status, credit_hist, purpose, saving_acct_status, employ_hist, sex&marital, other_debtors, property, installment_plans, housing, job, tel_provided, foreign_worker

**Continuous features:** 7 in total

Duration_in_month, credit_amount, installment_rate, residence_since, age, #existing_credits, #people_liable

To see detailed explanations of them, please refer to <https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)>

## 2. Load data

In [216]:
# Preprocessing: string -> integer
'''infile=open('german.data.txt','r')
outfile=open('german_numeric.csv','w')
for line in infile:
    items=line.split(' ')
    for item in items:
        if item[0]=='A':
            item='10' if item=='A410' else item[-1] #the last char is the index of the feature value, except 'A410'
        outfile.write(item+',')
    #outfile.write('')
infile.close()
outfile.close()'''

In [217]:
# Load data
import csv
import numpy as np
reader=csv.reader(open("german-numeric.csv","rt"),delimiter=',')
data=list(reader)
Xy=np.array(data).astype('int')
X=Xy[:,0:20]
y=Xy[:,20]
print('Data shape:\n%d features, %d instances' % (len(X[0]),len(X)))
print('\nHead 5 lines of X:')
print(X[:5])
y=-2*(y-1.5) # the y values are 1's and 2's, so we need to transform them to 1's and (-1)'s
print('\nHead 5 lines of y (1 = Good credit, -1 = Bad credit):')
print(y[:5])

Data shape:
20 features, 1000 instances

Head 5 lines of X:
[[   1    6    4    3 1169    5    5    4    3    1    4    1   67    3
     2    2    3    1    2    1]
 [   2   48    2    3 5951    1    3    2    2    1    2    1   22    3
     2    1    3    1    1    1]
 [   4   12    4    6 2096    1    4    2    3    1    3    1   49    3
     2    1    2    2    1    1]
 [   1   42    2    2 7882    1    4    2    3    3    4    2   45    3
     3    1    3    2    1    1]
 [   1   24    3    0 4870    1    3    3    3    1    4    4   53    3
     3    2    3    2    1    1]]

Head 5 lines of y (1 = Good credit, -1 = Bad credit):
[ 1. -1.  1.  1. -1.]


## 3. Feature binarization
Each continuous feature is binarized based on whether it is larger or smaller than its mean. Categorical features that have more than two possible values is binarized using one-hot encoding.

In [218]:
#apply one-hot encoding to categorical features

from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(categorical_features=[0,2,3,5,6,8,9,11,13,14,16,18,19])
X1=enc.fit_transform(X).toarray().astype(int)
print('\nHead 5 lines of new X:')
print(X1[:5])


Head 5 lines of new X:
[[   1    0    0    0    0    0    0    0    1    0    0    0    1    0
     0    0    0    0    0    0    0    0    0    1    0    0    0    0
     1    0    0    1    0    1    0    0    1    0    0    0    0    0
     1    0    1    0    0    0    1    0    0    1    1    0    6 1169
     4    4   67    2    1]
 [   0    1    0    0    0    0    1    0    0    0    0    0    1    0
     0    0    0    0    0    1    0    0    0    0    0    0    1    0
     0    0    1    0    0    1    0    0    1    0    0    0    0    0
     1    0    1    0    0    0    1    0    1    0    1    0   48 5951
     2    2   22    1    1]
 [   0    0    0    1    0    0    0    0    1    0    0    0    0    0
     0    1    0    0    0    1    0    0    0    0    0    0    0    1
     0    0    0    1    0    1    0    0    1    0    0    0    0    0
     1    0    1    0    0    1    0    0    1    0    1    0   12 2096
     2    3   49    1    2]
 [   1    0    0    0    0  

In [219]:
# binarize continuous features
continuous_feat=['Duration_in_month', 'credit_amount', 'installment_rate', 'residence_since', 'age', '#existing_credits', '#people_liable']
means=[]
X2=np.copy(X1)
for i in range(54,61): # the last 7 columns of X
    m=np.mean(X1[:,i])
    means.append(m)
    X2[X1[:,i]>m,i]=1 #if the value is greater than mean, then set it to 1
    X2[X1[:,i]<=m,i]=0

print('Mean values of each continuous feature:')
for i in range(len(means)):
    print(continuous_feat[i],means[i])
print('\nHead 5 lines of new X:')
print(X2[:5])

Mean values of each continuous feature:
Duration_in_month 20.903
credit_amount 3271.258
installment_rate 2.973
residence_since 2.845
age 35.546
#existing_credits 1.407
#people_liable 1.155

Head 5 lines of new X:
[[1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1
  0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0]
 [0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1
  0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1
  0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1]
 [1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0
  1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 1 1 0 1]
 [1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0
  0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1]]


In [220]:
feat_names=[
    'Chckg_acct_status:<0DM','Chckg_acct_status:<200DM','Chckg_acct_status:>200DM','Chckg_acct_status:no_acct',
    'credit_hist:0','credit_hist:1','credit_hist:2','credit_hist:3','credit_hist:4',
    'purpose:new_car','purpose:used_car','purpose:furniture','purpose:radio','purpose:appliance','purpose:repairs',
    'purpose:education','purpose:retraining','purpose:business','purpose:other',
    'saving_acct_status:<100DM','saving_acct_status:<500DM','saving_acct_status:<1000DM','saving_acct_status:>1000','saving_acct_status:no_acct',
    'employ_hist:unempl','employ_hist:1year','employ_hist:4years','employ_hist:7years','employ_hist:>7years',
    'sex&marital:M&divorced','sex&marital:F&divorced/married','sex&marital:M&single','sex&marital:M&married', #'sex&marital:F&single' doesn't exist
    'other_debtors:none','other_debtors:co-applicant','other_debtors:guarantor',
    'property:real_estate','property:life_insurance','property:car','property:none',
    'installment_plans:bank','installment_plans:stores','installment_plans:none',
    'housing:rent','housing:own','housing:free','job:unempl','job:unskilled','job:skilled','job:highly_qualified',
    'tel_provided:none','tel_provided:yes','foreign_worker:yes','foreign_worker:no',
    'Duration_in_month>20','credit_amount>3214.57','installment_rate>2.97','residence_since>2.85','age>35','#existing_credits>1.4','#people_liable>1'
]
print('%d binary features in total' % len(feat_names))
print('The shape of X2 is', X2.shape)
#for i in [0,2,3,5,6,8,9,11,13,14,16,18,19]:
#    print(len(set(X[:,i])))
#enc.n_values_

61 binary features in total
The shape of X2 is (1000, 61)


## 4. Model training

In [221]:
# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.333)
#X_train[:4]

In [222]:
# training
#http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html
from sklearn.naive_bayes import BernoulliNB
clf=BernoulliNB()
clf.fit(X_train,y_train)
ln_P_y=clf.class_log_prior_
ln_P_x_given_y=clf.feature_log_prob_
#print(np.exp(ln_P_x_given_y[0]))
print(clf.feature_count_[0])
#print(clf.class_count_)
print(np.where(np.logical_and(X_train[:,0]==1, y_train==-1))[0].shape)
print(np.where(np.logical_and(X_train[:,1]==1, y_train==-1))[0].shape)
print(np.where(np.logical_and(X_train[:,2]==1, y_train==-1))[0].shape)
print("Now I figured out 'feature_count_' is counting the occurrence of *positive* feature values")

[  87.   77.   10.   29.   14.   18.  121.   16.   34.   56.   14.   41.
   44.    3.    5.   11.    1.   24.    4.  144.   24.    8.    3.   24.
   15.   48.   74.   24.   42.   17.   76.   95.   15.  187.   11.    5.
   40.   44.   75.   44.   38.   13.  152.   51.  127.   25.    5.   37.
  126.   35.  121.   82.  200.    3.  120.   89.  135.  111.   70.   61.
   28.]
(87,)
(77,)
(10,)
Now I figured out 'feature_count_' is counting the occurrence of *positive* feature values


## 5. Transparency study 

In each of the following sections, we will study the transparency of classifying a selected object. Specifically we will check a) the total positive log-evidence,  b) the total negative log-evidence, c) probability distribution, d) top 3 features values that contribute most to the positive evidence, e) top 3 feature values that contribute the most to the negative evidence.

In [223]:
## Param: obj, an array of feature values
## Return: total log positive evidence, total log negative evidence, 
##   a list of log positive evidence shares contributed by each "yes" feature value, and 
##   a list of log negative evidence shares contributed by each "no" feature value
def compute_total_log_evidence(obj):
    log_pos_evidence=0
    log_neg_evidence=0
    contributions_yes_feat_values=[]
    contributions_no_feat_values=[]
    log_ratio_prior=ln_P_y[1]-ln_P_y[0]
    
    # add log[P(y=+1)/P(y=-1)]
    if log_ratio_prior>0: 
        log_pos_evidence+=log_ratio_prior
    elif log_ratio_prior<0:
        log_neg_evidence+=log_ratio_prior
    
    # sum up log [P(xi|y=+1)/P(xi|y=-1)]
    for i in range(len(obj)): 
        if obj[i]==1: #if feat value is 1, we can directly use the clf.feature_log_prob_
            ln_ratio=ln_P_x_given_y[1,i]-ln_P_x_given_y[0,i] 
        else: #otherwise, we need to do some calculations
            p=1-np.exp(ln_P_x_given_y[1,i])
            n=1-np.exp(ln_P_x_given_y[0,i])
            ln_ratio=np.log(p/n)
        if ln_ratio>0:
            log_pos_evidence+=ln_ratio
            contributions_yes_feat_values.append((i,ln_ratio))
        elif ln_ratio<0:
            log_neg_evidence+=ln_ratio
            contributions_no_feat_values.append((i,ln_ratio))
    
    return log_pos_evidence, log_neg_evidence, contributions_yes_feat_values, contributions_no_feat_values

### 1) The most positive object with respect to the probabilities

In [224]:
k=3 #top-k pos/neg feature values
prob_dists=clf.predict_proba(X_test)
#max_indices=np.argmax(prob_dists,axis=0)
most_pos=np.argmax(prob_dists[:,1])
print('The index of the most positive object is %d'% most_pos)
results=compute_total_log_evidence(X_test[most_pos])
print('a) the total positive log-evidence is %f' % results[0])
print('b) the total negative log-evidence is %f' % results[1])
print('c) the class probability distribution is', prob_dists[most_pos])

#find the indices of top 3 positive feature values based on the list 'contributions_yes_feat_values'
top3_pos_feat_idx_contrib=sorted(results[2], key=lambda x: -x[1])[:k] 
print('\nd) top %d features values that contribute most to the positive evidence:' % k)
for (i,contrib) in top3_pos_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_pos][i] else '✗','\t\t\t',contrib)

top3_neg_feat_idx_contrib=sorted(results[3], key=lambda x: x[1])[:k] 
print('\ne) top %d features values that contribute most to the negative evidence:' % k)
for (i,contrib) in top3_neg_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_pos][i] else '✗','\t\t\t',contrib)
    
print('\n ✓ represents feature=1, ✗ represents feature=0')

The index of the most positive object is 159
a) the total positive log-evidence is 9.294994
b) the total negative log-evidence is -1.445884
c) the class probability distribution is [  3.899467e-04   9.996101e-01]

d) top 3 features values that contribute most to the positive evidence:
Chckg_acct_status:no_acct ✓ 			 1.22866541692
saving_acct_status:>1000 ✓ 			 1.05062652191
foreign_worker:no ✓ 			 0.736968963059

e) top 3 features values that contribute most to the negative evidence:
credit_hist:4 ✗ 			 -0.233388990085
property:real_estate ✗ 			 -0.152721087018
saving_acct_status:no_acct ✗ 			 -0.142014716402

 ✓ represents feature=1, ✗ represents feature=0


### 2) The most negative object with respect to the probabilities

In [225]:
most_neg=np.argmax(prob_dists[:,0])
print('The index of the most negative object is %d'% most_neg)
results=compute_total_log_evidence(X_test[most_neg])
print('a) the total positive log-evidence is %f' % results[0])
print('b) the total negative log-evidence is %f' % results[1])
print('c) the class probability distribution is', prob_dists[most_neg])

#find the indices of top 3 positive feature values based on the list 'contributions_yes_feat_values'
top3_pos_feat_idx_contrib=sorted(results[2], key=lambda x: -x[1])[:k] 
print('\nd) top %d features values that contribute most to the positive evidence:' % k)
for (i,contrib) in top3_pos_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_neg][i] else '✗','\t\t\t',contrib)

top3_neg_feat_idx_contrib=sorted(results[3], key=lambda x: x[1])[:k] 
print('\ne) top %d features values that contribute most to the negative evidence:' % k)
for (i,contrib) in top3_neg_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_neg][i] else '✗','\t\t\t',contrib)

The index of the most negative object is 262
a) the total positive log-evidence is 2.868176
b) the total negative log-evidence is -8.194982
c) the class probability distribution is [ 0.995164  0.004836]

d) top 3 features values that contribute most to the positive evidence:
employ_hist:7years ✓ 			 0.503243302415
Chckg_acct_status:<0DM ✗ 			 0.346239553605
credit_amount>3214.57 ✗ 			 0.167233493547

e) top 3 features values that contribute most to the negative evidence:
credit_hist:1 ✓ 			 -1.36771936136
property:none ✓ 			 -0.657546231205
installment_plans:bank ✓ 			 -0.552911668393


### 3) The object that has the largest positive evidence

In [226]:
# first we compute the pos/neg evidence for all objects in the test set.
pos_evidence_of_all_obj=[]
neg_evidence_of_all_obj=[]
for obj in X_test:
    results=compute_total_log_evidence(obj)
    pos_evidence_of_all_obj.append(results[0])
    neg_evidence_of_all_obj.append(results[1])

In [227]:
idx=np.argmax(pos_evidence_of_all_obj)
print('The index of the object with the largest positive evidence is %d'% idx)
results=compute_total_log_evidence(X_test[idx])
print('a) the total positive log-evidence is %f' % results[0])
print('b) the total negative log-evidence is %f' % results[1])
print('c) the class probability distribution is', prob_dists[idx])

#find the indices of top 3 positive feature values based on the list 'contributions_yes_feat_values'
top3_pos_feat_idx_contrib=sorted(results[2], key=lambda x: -x[1])[:k] 
print('\nd) top %d features values that contribute most to the positive evidence:' % k)
for (i,contrib) in top3_pos_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[idx][i] else '✗','\t\t\t',contrib)

top3_neg_feat_idx_contrib=sorted(results[3], key=lambda x: x[1])[:k] 
print('\ne) top %d features values that contribute most to the negative evidence:' % k)
for (i,contrib) in top3_neg_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[idx][i] else '✗','\t\t\t',contrib)

The index of the object with the largest positive evidence is 159
a) the total positive log-evidence is 9.294994
b) the total negative log-evidence is -1.445884
c) the class probability distribution is [  3.899467e-04   9.996101e-01]

d) top 3 features values that contribute most to the positive evidence:
Chckg_acct_status:no_acct ✓ 			 1.22866541692
saving_acct_status:>1000 ✓ 			 1.05062652191
foreign_worker:no ✓ 			 0.736968963059

e) top 3 features values that contribute most to the negative evidence:
credit_hist:4 ✗ 			 -0.233388990085
property:real_estate ✗ 			 -0.152721087018
saving_acct_status:no_acct ✗ 			 -0.142014716402


### 4)The object that has the largest (in magnitude) negative evidence

In [228]:
idx=np.argmin(neg_evidence_of_all_obj)
print('The index of the object with the largest (in magnitude) negative evidence is %d'% idx)
results=compute_total_log_evidence(X_test[idx])
print('a) the total positive log-evidence is %f' % results[0])
print('b) the total negative log-evidence is %f' % results[1])
print('c) the class probability distribution is', prob_dists[idx])

#find the indices of top 3 positive feature values based on the list 'contributions_yes_feat_values'
top3_pos_feat_idx_contrib=sorted(results[2], key=lambda x: -x[1])[:k] 
print('\nd) top %d features values that contribute most to the positive evidence:' % k)
for (i,contrib) in top3_pos_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[idx][i] else '✗','\t\t\t',contrib)

top3_neg_feat_idx_contrib=sorted(results[3], key=lambda x: x[1])[:k] 
print('\ne) top %d features values that contribute most to the negative evidence:' % k)
for (i,contrib) in top3_neg_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[idx][i] else '✗','\t\t\t',contrib)

The index of the object with the largest (in magnitude) negative evidence is 199
a) the total positive log-evidence is 3.516410
b) the total negative log-evidence is -8.354158
c) the class probability distribution is [ 0.992137  0.007863]

d) top 3 features values that contribute most to the positive evidence:
purpose:radio ✓ 			 0.369374129007
Duration_in_month>20 ✗ 			 0.304835607869
sex&marital:M&single ✓ 			 0.249265756714

e) top 3 features values that contribute most to the negative evidence:
credit_hist:1 ✓ 			 -1.36771936136
Chckg_acct_status:<0DM ✓ 			 -0.798702799135
property:none ✓ 			 -0.657546231205


### 5) The most uncertain object (the probabilities are closest to 0.5)

In [229]:
most_uncertain=np.argmin(abs(prob_dists[:,0]-prob_dists[:,1]))
print('The index of the most uncertain object is %d'% most_uncertain)
results=compute_total_log_evidence(X_test[most_uncertain])
print('a) the total positive log-evidence is %f' % results[0])
print('b) the total negative log-evidence is %f' % results[1])
print('c) the class probability distribution is', prob_dists[most_uncertain])

#find the indices of top 3 positive feature values based on the list 'contributions_yes_feat_values'
top3_pos_feat_idx_contrib=sorted(results[2], key=lambda x: -x[1])[:k] 
print('\nd) top %d features values that contribute most to the positive evidence:' % k)
for (i,contrib) in top3_pos_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_uncertain][i] else '✗','\t\t\t',contrib)

top3_neg_feat_idx_contrib=sorted(results[3], key=lambda x: x[1])[:k] 
print('\ne) top %d features values that contribute most to the negative evidence:' % k)
for (i,contrib) in top3_neg_feat_idx_contrib:
    print(feat_names[i],'✓' if X_test[most_uncertain][i] else '✗','\t\t\t',contrib)

The index of the most uncertain object is 332
a) the total positive log-evidence is 4.410852
b) the total negative log-evidence is -4.432977
c) the class probability distribution is [ 0.505531  0.494469]

d) top 3 features values that contribute most to the positive evidence:
saving_acct_status:no_acct ✓ 			 0.669478721457
saving_acct_status:<100DM ✗ 			 0.496125834646
Chckg_acct_status:<0DM ✗ 			 0.346239553605

e) top 3 features values that contribute most to the negative evidence:
Chckg_acct_status:no_acct ✗ 			 -0.534923175345
housing:rent ✓ 			 -0.468354280364
Chckg_acct_status:<200DM ✓ 			 -0.459385610382
